#### Abstract : Predict the doctor fees


# Import Libraries and dataset

In [ ]:
# importing the libraries
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

# importing the warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#import the dataset
doctor = pd.read_excel("Final_Test.xlsx")

In [ ]:
#view the first five records
doctor.head()

### Observation:
- There are 7 columns and all the independent variables of type object
- The columns are mixed with different forms of data 

In [ ]:
#checking the shape of data
doctor.shape

### There are 5961 records and 7 columns
- After proper EDA and Data Cleaning the number of records and columns will change.

In [ ]:
# Checking the general information 
doctor.info()

### Observation

- There are null values in rating column
- There are null values in Miscellaneous_info

#### Fees - The Fees column is the target but the values seems to be discrete.
- All the independent variables are categorical, therefore supervised/classification model will be selected.

# Exploratory Data Analysis and Data Cleaning

In [ ]:
#check for null values
doctor.isnull().sum()

In [ ]:
# heatmap to inspect null values
sns.heatmap(doctor.isnull())

###### The percentage of data missing in Rating is 55.4% - cannot drop rows - simple imputer
###### The percentage of data missing in place is  4.2%  - can drop rows  - dropna
###### the percent of data missing in miscellaneous_info is 44% - cannot drop row - simple imputer

##### split each column to get clear understanding

In [ ]:
# simple imputer to remove null values
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy = "most_frequent")
doctor["Rating"] =imp.fit_transform(doctor["Rating"].values.reshape(-1,1))
doctor["Miscellaneous_Info"] =imp.fit_transform(doctor["Miscellaneous_Info"].values.reshape(-1,1))

In [ ]:
doctor.dropna(inplace = True)

In [ ]:
doctor.shape

In [ ]:
doctor.isnull().sum()

- Null values removed

##### Inspecting Each column seperately

## Quaification

In [ ]:
qualification =doctor[["Qualification"]].copy()

In [ ]:
qualification

In [ ]:
 qualification["Qualification"] = qualification["Qualification"].str.replace(",", " ").str.replace("-", "")

remove = ["'","/","(",")","&",".","smile","Year","In","in","And","AND","and",'VD','FAMS','Fellowship','of','the','Doctor','Science','Academy','Royal','College','Post', 'Graduate', 'Diploma','Crown','Bridge']

for r in remove:
    qualification["Qualification"] = qualification["Qualification"].str.replace(r, "")

In [ ]:
qualification

In [ ]:
#spliting the column
quali = qualification.Qualification.str.split().tolist()

In [ ]:
quali

In [ ]:
#create the dataframe with column
df_quali = pd.DataFrame(quali)

In [ ]:
df_quali

In [ ]:
df_quali = df_quali[[0,1,2]].copy()

In [ ]:
#Selecting the first three columns
df_quali

In [ ]:
#rename first three columns

df_quali.rename(columns ={0 : "UG", 1:"PG", 2:"Specialization"}, inplace=True)

In [ ]:
df_quali

In [ ]:
df_quali.isnull().sum()

In [ ]:
df_quali.shape

In [ ]:
print(df_quali["UG"].unique())


In [ ]:
df_quali["UG"].value_counts()


In [ ]:
ug = ['BHMS', 'BAMS', 'MBBS', 'BSc', 'BDS', 'MDS', 'Dermatology', 'DM', 'MD', 'MDSOral'
 ,'PhD', 'MRCPS', 'MS', 'DDVL', 'DHMS', 'GCEH', 'Healthcare', 'DNB',
 'Certification' ,'Laryngology', 'FAGE', 'Aestic', 'BSAM', 'Family', 'FNB'
, 'Dermotology' ,'Certificate' ,'Certified','LCEH', 'Homeopathic', 'Child'
, 'Otorholaryngology',  'MSc' ,  'Medical' ,'FCPS', 'GAMS',
 'BUMS' ,'Diet', 'MBA',  'PG', 'POST', 'Oral', 'Diabetology', 'MCh', 'DDV',
  'BEMS', 'MRCS', 'Periodontics', 'MA', 'MPH', 'MRCPCH', 'FRCS', 'BAc',
 'BIMS', 'Trichology',  'Ayurvedic', 'DHM',  'Diabetic',
 'MRCGP' ,'Naturopathy', 'MRCP', 'CGO', 'PGD', 'DNHE', 'FRGUHS' , 'FRCP',
 'Clical', 'Cochlear' ,'Masters' ,'PDCC' ,'FFDRCSI', 'MDDVL' ,'DGO']

In [ ]:
df = df_quali.loc[df_quali["UG"].isin(ug)]

In [ ]:
df

In [ ]:
df["UG"].value_counts()

In [ ]:
df.shape

In [ ]:
Qualification = df[["UG"]].copy()

In [ ]:
Qualification 

### Experience column

In [ ]:
experience = doctor[["Experience"]].copy()
experience = experience.Experience.str.split().tolist()
df_exp = pd.DataFrame(experience, columns=["Experience", 0, 1])
df_exp = df_exp[["Experience"]].copy()
df_exp["Experience"]= df_exp["Experience"].astype("int")

In [ ]:
df_exp.head()

### Rating Column

In [ ]:
rating = doctor[["Rating"]].copy()
rating["Rating"] = rating["Rating"].str.replace("%", "")
rating = rating.Rating.str.split().tolist()

df_rate = pd.DataFrame(rating, columns=["Rating"])
df_rate["Rating"] = df_rate["Rating"].astype(int)
df_rate.head()

### Place column

In [ ]:
place = doctor[["Place"]].copy()
place["Place"] = place["Place"].str.replace(" ", "")
place["Place"] = place["Place"].str.replace(",", " ").str.replace("-", "")
place = place.Place.str.split().tolist()
df_place = pd.DataFrame(place)
print(df_place[1].unique())
df_place[1] = df_place[1].replace("Sector5", "Delhi")
df_place = df_place[[0,1]].copy()
df_place.rename(columns = {0:"Area", 1:"City"}, inplace=True)
print(df_place.head())
place = df_place[["City"]].copy()

In [ ]:
df_place["City"].unique()

In [ ]:
place.head()

In [ ]:
place.isnull().sum()

In [ ]:
place  = place.fillna(df_place.mode().iloc[0])

#### Miscellaneous_Info

In [ ]:
M_info  = doctor[["Miscellaneous_Info"]]
M_info = M_info.Miscellaneous_Info.str.split().tolist()
df_m = pd.DataFrame(M_info)
df_m = df_m[[0]].copy()
df_m.rename(columns = {0:"comments"}, inplace = True)
c = df_m["comments"]
# using regex to find the comments
c.replace(r'^[0-9][0-9]%', "Unknown", regex=True, inplace=True)
c.replace(r'^[0-9]$', "Unknown", regex=True, inplace=True)
c.replace(r'^[7]%', "Unknown", regex=True, inplace=True)
c.replace("100%", "Unknown", inplace=True)

In [ ]:
c.unique()

In [ ]:
df_mis = c.to_frame()


In [ ]:
profile = doctor[['Profile']].copy()

#### Column to be joined using new dataframes
- Qualification 
- Place
- df_exp
- df_mis
- df_Rate
- Profile


#### concat the dataframes one at a time

In [ ]:
doc = pd.DataFrame()
li = [Qualification,place,df_exp,df_mis,profile]

doc = pd.concat([Qualification,place], axis=1, join="inner")

In [ ]:
doc

In [ ]:
doc_one = pd.concat([doc,df_exp], axis=1, join="inner")

In [ ]:
print(doc_one.shape)
doc_one.head()

In [ ]:
doc_two = pd.concat([doc_one,df_mis], axis=1, join="inner")

In [ ]:
doc_two

In [ ]:
Doc_the = pd.concat([doc_two,profile], axis=1, join="inner")

In [ ]:
Doc_the.head()

In [ ]:
Doc =  pd.concat([Doc_the,df_rate], axis=1, join="inner")

In [ ]:
Doc.head()

# Univariate analysis

In [ ]:
#count plot for categorical variable 
fig_size = (20,15)
fig, ax = plt.subplots(figsize = fig_size)
sns.countplot(data=Doc, x="UG")
plt.xticks(rotation=90)
plt.yticks(fontsize=50)
plt.xlabel("UG",fontsize=80)
plt.ylabel("Count",fontsize=80)
plt.title("UG count", fontsize = 80)
Doc["UG"].value_counts()

### MBBS seems to have the Highest count in qualification

In [ ]:
#count plot for categorical variable 
fig_size = (25,15)
fig, ax = plt.subplots(figsize = fig_size)
sns.countplot(data=Doc, x="City")
plt.xticks(rotation=90, fontsize=50)
plt.yticks(fontsize=50)
plt.xlabel("City",fontsize=50)
plt.ylabel("Count",fontsize=50)
plt.title("City count", fontsize = 50)
Doc["City"].value_counts()

#### The Banglore population count is highest of all, followed by mumbai at 1207, delhi at 1175, Hyderbad at 948 and so on

In [ ]:
#count plot for categorical variable 
fig_size = (35,30)
fig, ax = plt.subplots(figsize = fig_size)
sns.countplot(data=Doc, x="Profile")
plt.xticks(rotation=90, fontsize=50)
plt.yticks(fontsize=50)
plt.xlabel("Profile",fontsize=50)
plt.ylabel("Count",fontsize=50)
plt.title("Profile count", fontsize = 50)
Doc["Profile"].value_counts()

### dentist have the highest count of all followed by General medicine and deramtologists. ENT specialist have the lowest count

In [ ]:
#count plot for categorical variable 
fig_size = (35,30)
fig, ax = plt.subplots(figsize = fig_size)
sns.countplot(data=Doc, x="Rating")
plt.xticks(rotation=90, fontsize=50)
plt.yticks(fontsize=50)
plt.xlabel("Rating",fontsize=50)
plt.ylabel("Count",fontsize=50)
plt.title("Rating count", fontsize = 50)
Doc["Rating"].value_counts()

#### Most of the Doctors are rated to give 100% satisfaction and service 

# Multivariate Analysis

## Summary statistics

In [ ]:
Doc.describe()

###  Since other independent variables are categorical they are not included in the summary stat
- huge difference in the range  - needs scalling
- huge difference between mean and standard deviation

### Correlation Matrix

In [ ]:
correlation = Doc.corr()
plt.figure(figsize = [15,10])
sns.heatmap(correlation, annot=True)
plt.title("Correlation Matric Doctor fees prediction")
plt.show()

### Label encoding

In [ ]:
# import the labelencoder
from sklearn.preprocessing import LabelEncoder

# initialize Labelencoder
LE = LabelEncoder()

Doc["UG"] = LE.fit_transform(Doc["UG"])
    

In [ ]:
Doc

# ONE HOT ENCODING

In [ ]:
#Introducing one hot encoder to convert categorical feature 
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
toen = ["City", "Profile"]

new2 = ohe.fit_transform(Doc["City"].values.reshape(-1,1)).toarray()
new_df2 = pd.DataFrame(new2)

new = ohe.fit_transform(Doc["Profile"].values.reshape(-1,1)).toarray()
new_df = pd.DataFrame(new, columns = [np.unique(Doc["Profile"])])



In [ ]:
new_df

In [ ]:
#concat encoded profile with data frame
Doc = pd.concat([Doc, new_df], join = "inner", axis = 1)

In [ ]:
Doc.head()


- successfully joined

In [ ]:
#drop the orginal column
Doc = Doc.drop("Profile", axis=1)

In [ ]:
Doc.head()

- Succefully dropped the profile column

In [ ]:
#join encoded city column with Doc
Doc = pd.concat([Doc, new_df2], join = "inner", axis = 1)

In [ ]:
Doc.head()

- columns added

In [ ]:
#drop city
Doc 

In [ ]:
Doc = Doc.drop("City", axis=1)

- dropped city column

In [ ]:
Doc.head()

#### Handling Rating column

##### categorizing the rating column for easy of prediction

In [ ]:
#apply where 
Doc["Rating"] = np.where(Doc["Rating"]<90,0, Doc["Rating"])
Doc["Rating"] = np.where(Doc["Rating"]>=90,1, Doc["Rating"])


- check the column for unique values

In [ ]:
#apply unique for series
Doc["Rating"].unique()

In [ ]:
#comments seems to be inapproriate for the prediction
Doc = Doc.drop("comments", axis=1)

In [ ]:
Doc.head()

In [ ]:
Doc["Experience"].unique()

In [ ]:
Doc["Experience"] = np.where(Doc["Experience"]<=25,0, Doc["Experience"])
Doc["Experience"] = np.where(Doc["Experience"]>25,1, Doc["Experience"])

In [ ]:
Doc

- The Label Encoding and One hot encoding is applied and the data is ready for next step

In [ ]:
Doc.shape

### There are 5852 records and 18 columns

In [ ]:
Doc.describe()

- Huge difference between mean and standard deviation
- Huge difference in range

In [ ]:
correlation = Doc.corr()
plt.figure(figsize = [15,10])
sns.heatmap(correlation, annot=True)
plt.title("Correlation Matric Doctor fees prediction")
plt.show()

In [ ]:
Doc.shape

### There are 5852 records and 18 columns

# Check for Skewness and removal

- Note:  All the independent variables are of categorical data type hence the skewness will not be involved.

# Check for Outlier and Removal

In [ ]:
#plot boxplot
Doc.boxplot(figsize = [25,15])

##### no outliers

In [ ]:
Doc.dtypes

In [ ]:
#scaling standardize the model thus the prediction come out with accuracy
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()

Doc = scale.fit_transform(Doc)

#scaled 

In [ ]:
import gzip, pickle
with gzip.open("Doctorfeefinal.pklz", 'rb') as ifp:
    fees = pickle.load(ifp)

In [ ]:
import joblib
fee = joblib.load("Doctorfeefinal.pklz")

In [ ]:
fee